In [47]:
%load_ext autoreload
%autoreload 2
import jieba
import transformers as ts
import pandas as pd
import numpy as np
import torch
import json
from sklearn.model_selection import train_test_split
from ark_nlp.model.ner.global_pointer_bert import (Dataset,Tokenizer,GlobalPointerBertConfig,
                                                   GlobalPointerBert,get_default_model_optimizer,Task,Predictor)
from model import GlobalPointer,GlobalPointerCrossEntropy,GlobalPointerNERPredictor
from torch.utils.data import Dataset,DataLoader
import os
import logging
import torch.nn as nn
import copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
label_list=['C-对比选项',
 'C-研究方法',
 'C-研究目的',
 'I-其他干预',
 'I-干预',
 'I-教育/行为干预',
 'I-药物干预',
 'I-非药物干预',
 'O-定性结论',
 'O-定量结论',
 'O-结论',
 'O-结论主语',
 'P-人群/患者类型',
 'P-条件',
 'P-研究对象',
 'P-评估项',
 'S-因素(病因/风险)分析',
 'S-指南标准建议',
 'S-治疗',
 'S-病因学',
 'S-统计分析',
 'S-诊断',
 'S-预后']

cat2id={'C-对比选项': 0,
 'C-研究方法': 1,
 'C-研究目的': 2,
 'I-其他干预': 3,
 'I-干预': 4,
 'I-教育/行为干预': 5,
 'I-药物干预': 6,
 'I-非药物干预': 7,
 'O-定性结论': 8,
 'O-定量结论': 9,
 'O-结论': 10,
 'O-结论主语': 11,
 'P-人群/患者类型': 12,
 'P-条件': 13,
 'P-研究对象': 14,
 'P-评估项': 15,
 'S-因素(病因/风险)分析': 16,
 'S-指南标准建议': 17,
 'S-治疗': 18,
 'S-病因学': 19,
 'S-统计分析': 20,
 'S-诊断': 21,
 'S-预后': 22}
id2cat={0: 'C-对比选项',
 1: 'C-研究方法',
 2: 'C-研究目的',
 3: 'I-其他干预',
 4: 'I-干预',
 5: 'I-教育/行为干预',
 6: 'I-药物干预',
 7: 'I-非药物干预',
 8: 'O-定性结论',
 9: 'O-定量结论',
 10: 'O-结论',
 11: 'O-结论主语',
 12: 'P-人群/患者类型',
 13: 'P-条件',
 14: 'P-研究对象',
 15: 'P-评估项',
 16: 'S-因素(病因/风险)分析',
 17: 'S-指南标准建议',
 18: 'S-治疗',
 19: 'S-病因学',
 20: 'S-统计分析',
 21: 'S-诊断',
 22: 'S-预后'}

In [3]:
tokenizer=ts.BertTokenizer.from_pretrained('../../model/macbert-base-chinese-medical-collation/')
ttokenier=Tokenizer(vocab=tokenizer,max_seq_len=200)

In [51]:
args={
        'bert_dir':'../../model/macbert-base-chinese-medical-collation/',
        'last_4_bert':True,
        'use_bilstm':True
    }
titlem=GlobalPointer(args,len(label_list),64)
tsd=torch.load('./title_best(lstm).pth')
titlem.load_state_dict(tsd)
title_ner=GlobalPointerNERPredictor(titlem,ttokenier,cat2id,tokenizer)

Some weights of BertModel were not initialized from the model checkpoint at ../../model/macbert-base-chinese-medical-collation/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
bcfg=ts.BertConfig.from_pretrained('../../model/macbert-base-chinese-medical-collation/',num_labels=len(label_list))
classfier=ts.BertForSequenceClassification.from_pretrained('../../model/macbert-base-chinese-medical-collation/',
                                                           config=bcfg,ignore_mismatched_sizes=True)
csd=torch.load('./best_classfier.pth')
classfier.load_state_dict(csd)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../../model/macbert-base-chinese-medical-collation/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../../model/macbert-base-chinese-medical-collation/ and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([23, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([23]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [53]:
title_ner.predict_one_sample("循证护理实践规范抗凝剂皮下注射对脑卒中患者发生皮下出血的影响及其影响因素研究")

[{'start_idx': 0, 'end_idx': 4, 'entity': '循证护理实', 'type': 'I-其他干预'}]

In [16]:
test=tokenizer("循证护理实践规范抗凝剂皮下注射对脑卒中患者发生皮下出血的影响及其影响因素研究",max_length=200,padding='max_length',)
test

{'input_ids': [101, 2542, 6395, 2844, 4415, 2141, 6664, 6226, 5745, 2834, 1125, 1177, 4649, 678, 3800, 2198, 2190, 5554, 1293, 704, 2642, 5442, 1355, 4495, 4649, 678, 1139, 6117, 4638, 2512, 1510, 1350, 1071, 2512, 1510, 1728, 5162, 4777, 4955, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [21]:
def reverse_tensor(text):
    input_ids=torch.LongTensor(text['input_ids']).view(1,-1)
    attention_mask=torch.LongTensor(text['attention_mask']).view(1,-1)
    token_type_ids=torch.LongTensor(text['token_type_ids']).view(1,-1)
    return {'input_ids':input_ids,'attention_mask':attention_mask,'token_type_ids':token_type_ids}

In [23]:
tttt=reverse_tensor(test)
tttt['input_ids'].size()

torch.Size([1, 200])

In [43]:
def clf(text):
    token=tokenizer(text,max_length=200,padding='max_length')
    token=reverse_tensor(token)
    with torch.no_grad():
        logits=classfier(**token)
    return id2cat[nn.Softmax(dim=-1)(logits['logits']).argmax().item()]

In [50]:
with open('./datasets/test.json',encoding='utf-8') as f:
    test_dict=json.load(f)
test_dict

[{'text': '常染色体显性多囊肾病的临床问题及其肾脏替代治疗的选择',
  'ner_list': [],
  'type': 'Title',
  'doc_id': 500},
 {'text': '',
  'ner_list': [{'mention': '表明,在体外条件下,胃癌干细胞对5-氟尿嘧啶敏感性较低,推测其可能为临床化疗耐药的重要机制。',
    'label': ''},
   {'mention': '体外环境下分析胃癌干细胞对5-氟尿嘧啶的敏感性,了解胃癌化疗耐药相关机制。', 'label': ''},
   {'mention': '基于克隆形态的分选策略,从人胃癌AGS细胞系内分离胃癌干细胞克隆,采用免疫细胞化学染色分析不同克隆CD44和胸苷酸合成酶的表达,克隆形成实验评估不同类型克隆的自我更新能力,CCK-8法检测不同浓度5-氟尿嘧啶作用下人胃癌AGS细胞克隆生长抑制率',
    'label': ''},
   {'mention': '与结论:人胃癌AGS细胞经低密度接种培养后,可形成32个不同形态的克隆,其中,副克隆、次克隆、全克隆所占比例分别为19%(6/32)、66%(21/32)、16%(5/32)。全克隆高表达CD44和胸苷酸合成酶,接种后可再次形成大量二代克隆;次克隆弱表达CD44和胸苷酸合成酶,接种后形成少量的二代克隆;副克隆不表达或弱表达CD44和胸苷酸合成酶,接种后未形成二代克隆。在不同浓度5-氟尿嘧啶的作用下,次克隆以及人胃癌AGS细胞的生长抑制率均显著高于全克隆(P均<0.05',
    'label': ''}],
  'type': 'Abstract',
  'doc_id': 501},
 {'text': '胃癌干细胞对5-氟尿嘧啶的敏感性', 'ner_list': [], 'type': 'Title', 'doc_id': 501},
 {'text': '',
  'ner_list': [{'mention': '96例患者中14例(14.6%)发生皮下出血。不同性别、年龄、体质指数、腹部皮肤褶皱厚度及联合使用抗凝剂患者皮下出血发生率比较,差异有统计学意义(P<0.05);不同文化程度、体位、纤维蛋白原、D-二聚体及有无抗凝剂用药史、使用腰带、出现其他部位出血患者皮下出血发

In [60]:
test_dict[790]

{'text': '',
 'ner_list': [{'mention': '通滞苏润江胶囊联合托珠单抗治疗类风湿性膝关节炎取得满意临床疗效,能够显著改善患者临床症状,降低患者炎性反应,不良反应小,具有良好的应用价值,值得临床推广使用',
   'label': ''},
  {'mention': '治疗后观察组总有效率93.33%高于对照组的75.56%(x^2=4.145,P=0.042);治疗后观察组晨僵时间为(1.24±0.45)h、压痛关节数为(3.6±1.8)个、肿胀关节数(4.8±1.4)个低于对照组(1.87±0.47)h、压痛关节数为(8.2±2.4)个、肿胀关节数(7.6±1.6)个(t=6.495、10.286、8.835,P<0.05);观察组ESR为(30.6±12.7)mm/h,CRP为(14.7±2.4)mg/L,DSA28为(2.16±0.87)分低于对照组(43.1±11.5)mm/h、(18.9±3.5)mg/L、(3.47±0.94)分(t=4.894、6.639、6.861,P<0.05);治疗后观察组患者IL-6为(25.12±13.45)ng/ml,IL-17为(15.36±4.42)pg/ml,IL-23为(301.48±51.84)pg/ml低于对照组(41.87±22.47)ng/ml,(24.62±4.94)pg/ml,(677.46±127.46)pg/ml(t=4.291、9.371.、17.842,P<0.05);观察组CCP为(193.56±25.34)IU/ml,RF为(87.25±25.67)IU/ml,AKA阳性率为22.22%低于对照组(242.35±32.54)IU/ml,(125.68±35.24)ILU/ml,42.22%(x^2=7.936、5.913、4.121,P<0.05);观察组药物不良反应为6.67%(3/45)高于对照组4.44%(2/45),但差异无统计学意义(x^2=0.218,P=0.645)。',
   'label': ''},
  {'mention': '观祭通滞苏润江股囊联合托珠单抗治疗类风湿性膝关节炎的效果及对IL-17、IL-23的影响。', 'label': ''},
  {'mention': '选取2012年6月—2014年6月宜昌市第一人民医院风湿

In [67]:
from tqdm import tqdm
submit_dict=[]
total=len(test_dict)
with tqdm(total=total) as pbar:
    for d in test_dict:
        newd=copy.deepcopy(d)
        if newd['type'] == 'Title':
            #分类
            title_type=clf(newd['text'])
            newd['ner_list'].append({
                'mention':newd['text'],
                'label':title_type,
            })
            #抽取
            entities=title_ner.predict_one_sample(newd['text'])
            for entity in entities:
                newd['ner_list'].append({
                    'mention':entity['entity'],
                    'label':entity['type'],
                })
        if newd['type'] == 'Abstract':
            #分类
            for ner in newd['ner_list']:
                cur_text=ner['mention']
                if len(cur_text) > 180:
                    todel=len(cur_text)-180
                    l=todel//2
                    r=todel-l
                    cur_text=cur_text[l:-r]
                sentence_type=clf(cur_text)
                ner['label']=sentence_type
        submit_dict.append(newd)
        pbar.update(1)

submit_json=json.dumps(submit_dict,ensure_ascii = False)
with open('./submits.json','w',encoding='utf-8') as f:
    f.write(submit_json)
    f.close()

100%|██████████| 3982/3982 [1:28:09<00:00,  1.33s/it]


In [65]:
submit_dict=json.dumps(submit_dict,ensure_ascii = False)
with open('./submits.json','w',encoding='utf-8') as f:
    f.write(submit_dict)
    f.close()